In [1]:
import sys
import json, os, re, shutil, sys, time
import collections, itertools
import pickle

from random import randint

from gensim.models import word2vec, KeyedVectors
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_recall_fscore_support

import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("0.12"))

interim_path = '../../interim/'

import rnnlm
import util

reload(rnnlm)
reload(util)

print "Done!"

Done!


In [2]:
samp = True
samp = '_samp' if samp else ''

In [3]:
# load preprocessed data array
with open( interim_path + 'preprocessed_concatenated_all' + samp + '.p') as p:
	podcast_array = pickle.load(p)

print podcast_array.shape

(1241, 3)


In [4]:
w2vmodel = KeyedVectors.load_word2vec_format( interim_path + 'GoogleNews-vectors-negative300.bin', binary=True)
w2vmodel.most_similar('dog')

[(u'dogs', 0.8680489659309387),
 (u'puppy', 0.8106428384780884),
 (u'pit_bull', 0.780396044254303),
 (u'pooch', 0.7627377510070801),
 (u'cat', 0.7609456777572632),
 (u'golden_retriever', 0.7500902414321899),
 (u'German_shepherd', 0.7465174198150635),
 (u'Rottweiler', 0.7437614798545837),
 (u'beagle', 0.7418621778488159),
 (u'pup', 0.740691065788269)]

In [5]:
#w2vmodel.save('../interim/gensim_model')
#w2vmodel = KeyedVectors.load('../interim/gensim_model')
w2vmodel['dog'].shape

(300,)

In [42]:
# ALTERNATIVE 1
# concatenate a maximum of 1000, 300-element word vectors per description
# into a single 300k element document vector
document_vectors = []
for podcast in podcast_array:
    # handle the edge case where there is no descrption
    if len(podcast[1]) == 0:
        podcast[1] = ''
    doc_vec = {}
    count = 0
    for i in range(0, min(999, len(podcast[1]))):
        word = podcast[1][i]
        if word in w2vmodel:
            if len(doc_vec) == 0:
                doc_vec = w2vmodel[word]
            else:
                doc_vec = np.concatenate((doc_vec, w2vmodel[word]), axis=0)
    document_vectors.append(doc_vec)

# save the document vectors    
filepath=interim_path + 'preprocessed_concatenated_document_vectors' + samp + '.p'
with open(filepath, 'wb') as f:
    pickle.dump(document_vectors,f)

In [4]:
# ALTERNATIVE 1 LOAD FROM FILE
filepath = interim_path + 'preprocessed_concatenated_document_vectors' + samp + '.p'
with open(filepath, 'rb') as f:
    new_doc_vector_list = pickle.load(f)
print len(new_doc_vector_list)

1241


In [6]:
# ALTERNATIVE 2
# take the average of all of the word vectors for each description
# treat each description as a document and generate a single vector for each
# Training parameters
size = 300
document_vectors = []
for podcast in podcast_array:
    word_vecs=[]
    for i in range(len(podcast[1])):
        word = podcast[1][i]
        if word in w2vmodel:
            word_vecs += w2vmodel[word].tolist()
    data = np.asarray(word_vecs).reshape(-1, size)
    document_vectors.append(np.average(data, axis=0))

# save the document vectors    
filepath=interim_path + 'preprocessed_averaged_document_vectors' + samp + '.p'
with open(filepath, 'wb') as f:
    pickle.dump(document_vectors,f)    

/home/michaelrubin/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/michaelrubin/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [4]:
# ALTERNATIVE 2 LOAD FROM FILE
filepath = interim_path + 'preprocessed_averaged_document_vectors' + samp + '.p'
with open(filepath, 'rb') as f:
    new_doc_vector_list = pickle.load(f)
print len(new_doc_vector_list)

1241


In [5]:
# break out train and test 
vl = np.array(new_doc_vector_list)
x_train, x_dev, y_train, y_dev = train_test_split(vl, podcast_array[:,[1]], test_size=0.25, random_state=42)
print x_train.shape, x_dev.shape, y_train.shape, y_dev.shape

(930, 300) (311, 300) (930, 1) (311, 1)


In [78]:
# training parameters
max_time = 20
batch_size = 10
learning_rate = 0.5
num_epochs = 5

# Model parameters
V=len(new_doc_vector_list)
H=len(new_doc_vector_list[0]) #*2
model_params = dict(V=V, H=H, softmax_ns=200, num_layers=1)

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [77]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        #### YOUR CODE HERE ####
        feed_dict = { lm.input_w_: w, 
                      lm.target_y_: y, 
                      lm.initial_h_ : h, 
                      lm.learning_rate_ : learning_rate, 
                      lm.use_dropout_ : use_dropout 
                    }
        
        h, cost, _ = session.run( [lm.final_h_, loss, train_op], feed_dict=feed_dict )
        
        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print "[batch %d]: seen %d vectors at %d vps, loss = %.3f" % (
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [79]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.batch_generator(ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=True, tick_s=3600)
    print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))

In [80]:
reload(rnnlm)
reload(util)

# Will print status every this many seconds
print_interval = 15

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in xrange(1,num_epochs+1):
        t0_epoch = time.time()
        bi = util.batch_generator(x_train, batch_size, max_time)
        print "[epoch %d] Starting epoch %d" % (epoch, epoch)
        
        run_epoch(lm, session, bi,
              train=True, verbose=True,
              tick_s=print_interval, learning_rate=learning_rate)
        
        print "[epoch %d] Completed in %s" % (epoch, util.pretty_timedelta(since=t0_epoch))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, x_train, name="Train set")
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, x_dev, name="Test set")
        print ""
    
    # Save final model
    saver.save(session, trained_filename)

[epoch 1] Starting epoch 1
[batch 178]: seen 35800 vectors at 2386 vps, loss = 0.047
[batch 353]: seen 70800 vectors at 2353 vps, loss = 0.024
[batch 528]: seen 105800 vectors at 2343 vps, loss = 0.016


InvalidArgumentError: indices[100] = -2147483648 is not in [0, 1241)
	 [[Node: per_example_sampled_softmax_loss/embedding_lookup_1 = Gather[Tindices=DT_INT64, Tparams=DT_FLOAT, _class=["loc:@recurrent_layer/Variable"], validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](recurrent_layer/Variable/read, per_example_sampled_softmax_loss/concat)]]

Caused by op u'per_example_sampled_softmax_loss/embedding_lookup_1', defined at:
  File "/home/michaelrubin/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/michaelrubin/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/michaelrubin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/michaelrubin/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/michael

In [75]:
reload(util)
'''
Function to compute precision, recall and f-score using knn and cosine similarity
Finds the k nearest neighbors ...
y_ is a vector of labels of length d
x_arr is a matrix of vectors of length d, width v
'''
def scoreVectors(y_, x_arr):
    k=10
    y_pred=[]
    y_true=[]
    result=[]
    for i in range(0, x_arr.shape[0]):
        true_label = y_[i]
        cnt = 0.0
        found = 0.0
        vec = x_arr[i]
        r_idx, r_val = util.find_nn_cos(vec, x_arr, k)
        
        for j in range(0, r_idx.shape[0]):
            if r_val[j] != 1.0:
                cnt +=1
                y_true+=true_label
                if(y_[r_idx[j]]==true_label):
                    y_pred+=y_[r_idx[j]]
                    found+=1
        if cnt > 0:
            result.append(found / cnt)
    score = np.average(result)
    print precision_recall_fscore_support(y_true, y_pred, average='binary')
    print "Score: %1.4f" % score
    return score
            

r  = scoreVectors( y_train, x_train )


(0.0, 0.0, 0.0, 0)
Score: 0.0399
